In [36]:
import csv
import pandas as pd
import pandas.io.gbq as bq
import numpy as np

In [ ]:
clav_names = '''Longliners
Tuna longliners
Fish carriers
Purse seiners
Trawlers
Tuna purse seiners
Pole and Line vessels
Non-fishing vessels nei
Unknown
Trollers
Recreational fishing vessels
Multipurpose vessels
Bunkers
Multipurpose vessels nei
Fishery research vessels
Support Vessels
Gillnetters
Handliners
Dredgers

Liners
Motherships
Freezer longliners
Liners nei
Seiners
Other fishing vessels
Seiners nei
Fishing vessels not specified
Other line vessels
Stern trawlers'''

In [26]:
clav_names_to_map = '''Longliners:Drifting_longlines
Tuna longliners:Drifting_longlines
Fish carriers:Reefer
Purse seiners:Purse_seines
Trawlers:Trawlers
Tuna purse seiners:Purse_seines
Pole and Line vessels:Pole_and_line
Non-fishing vessels nei:Unknown
Unknown:Unknown
Trollers:Trollers
Recreational fishing vessels:Recreational_fishing
Multipurpose vessels:Recreational_fishing
Bunkers:Bunker
Multipurpose vessels nei:Unknown
Fishery research vessels:Research
Support Vessels:Unknown
Gillnetters:Set_gillnets
Handliners:Handliners
Dredgers:Trawlers
:Unknown
Liners:Unknown
Motherships:Unknown
Freezer longliners:Unknown
Liners nei:Unknown
Seiners:Unknown
Other fishing vessels:Unknown
Seiners nei:Unknown
Fishing vessels not specified:Unknown
Other line vessels:Unknown
Stern trawlers:Trawlers'''.split("\n")

In [27]:
clav_mapping_old = {
    'Gillnetters':'Set_gillnets',
    'Longliners':'Drifting_longlines',
    'Pole and Line vessels':'Pole_and_line',
    'Purse seiners':"Purse_seines",
    'Recreational fishing vessels':"Recreational_fishing",
    'Seiners':'Purse_seines',
    'Seiners nei':"Purse_seines",
    'Strn trawlers':'Trawlers',
    'Tralwers':'Trawlers',
    'Trollers':'Trollers',
    'Tuna longliners':'Drifting_longlines',
    'Tuna purse seiners':"Purse_seines"
}


In [28]:
clav_mapping = {}
for c in clav_names_to_map:
    c = c.split(":")
    clav_mapping[c[0]]=c[1]
    

In [29]:
# The following are mappings that we figured out earlier --
# see ../kroodsma_listbuild/fishing_vessels_for_net_20161111.ipynb


#   WHEN shiptype = 'Tuna purse seiners' THEN 'Purse seines'
#   WHEN shiptype = 'Purse seiners' THEN 'Purse seines'
#   WHEN shiptype = 'Tuna longliners' THEN 'Drifting longlines'
#   WHEN shiptype = 'Longliners' then 'Drifting longlines'
#   WHEN shiptype = 'Trawler' THEN 'Trawlers'
#   WHEN shiptype = 'Liners' THEN 'Drifting longlines'
#   ELSE shiptype

In [30]:
# for now, we are treating all the weird stuff as "unkown" from the clav list

case_query = 'Case\n'
for c in clav_mapping:
    case_query += "     When shiptype='{0}' then '{1}'\n".format(c,clav_mapping[c])
case_query +="    else 'Unknown' end\n"

q = '''
#select mmsi, 
#if(number_shiptypes>1,Null,shiptype) shiptype,
#if(length_stddev > .15*length, null, length) length,
#if(tonnage_stddev > .15*tonnage, null, tonnage) tonnage
#from

#(
select mmsi, 
group_concat(shiptype,"|") shiptype, 
avg(length) length ,avg(tonnage) tonnage, 
avg(stddev_length) length_stddev, 
avg(stddev_tonnage) tonnage_stddev, 
count(*) number_shiptypes ,
from
(select a.mmsi as mmsi, a.shiptype as shiptype, avg(b.length) length, avg(c.tonnage) tonnage, 
stddev(b.length) stddev_length, stddev(c.tonnage) stddev_tonnage
from 
(SELECT mmsi, 
{case_query}
shiptype, 
clav_TUVI FROM [world-fishing-827:CLAV_match_results.v10_results]) a
left join
(select length, integer(clav_TUVI) clav_TUVI from [scratch_bjorn.CLAV_2016_Aug_04_ISSF_IMO] where length is not null)  b
on a.clav_TUVI = b.clav_TUVI
left join 
(select tonnage, integer(clav_TUVI) clav_TUVI from [scratch_bjorn.CLAV_2016_Aug_04_ISSF_IMO] where tonnage is not null)  c
on a.clav_TUVI = c.clav_TUVI
group by mmsi, shiptype)
group by mmsi
#)
'''.format(case_query=case_query)

In [31]:
print q


#select mmsi, 
#if(number_shiptypes>1,Null,shiptype) shiptype,
#if(length_stddev > .15*length, null, length) length,
#if(tonnage_stddev > .15*tonnage, null, tonnage) tonnage
#from

#(
select mmsi, 
group_concat(shiptype,"|") shiptype, 
avg(length) length ,avg(tonnage) tonnage, 
avg(stddev_length) length_stddev, 
avg(stddev_tonnage) tonnage_stddev, 
count(*) number_shiptypes ,
from
(select a.mmsi as mmsi, a.shiptype as shiptype, avg(b.length) length, avg(c.tonnage) tonnage, 
stddev(b.length) stddev_length, stddev(c.tonnage) stddev_tonnage
from 
(SELECT mmsi, 
Case
     When shiptype='' then 'Unknown'
     When shiptype='Handliners' then 'Handliners'
     When shiptype='Trawlers' then 'Trawlers'
     When shiptype='Unknown' then 'Unknown'
     When shiptype='Longliners' then 'Drifting_longlines'
     When shiptype='Tuna purse seiners' then 'Purse_seines'
     When shiptype='Seiners' then 'Unknown'
     When shiptype='Fish carriers' then 'Reefer'
     When shiptype='Multipurpose vessels'

In [32]:
# Because bigquery doesn't let us link up when the list is in Google drive, I downloaded the
# results of this query to from_bigquery/clav_v10_downloaded.csv

In [33]:
df = pd.read_csv('from_bigquery/clav_v10_downloaded.csv', comment='#')
df.head()

,mmsi,shiptype,length,tonnage,length_stddev,tonnage_stddev,number_shiptypes
0,338793000,Purse_seines,63.12,1446.500000,0.0,75.367670,1
1,416002947,Drifting_longlines,29.70,98.000000,0.0,0.000000,1
2,416000961,Drifting_longlines,23.20,61.230000,NaN,NaN,1
3,224654000,Drifting_longlines,40.50,434.223333,0.0,89.719106,1
4,412371206,Drifting_longlines,43.20,560.000000,0.0,0.000000,1


In [34]:
# if stdv of length > 20 percent the length of the vessel, we should through out the length
df[(df["length_stddev"] > df["length"]/5)]#['length']=np.nan

,mmsi,shiptype,length,tonnage,length_stddev,tonnage_stddev,number_shiptypes
37,356567000,Reefer|Unknown,7289.170000,12413.0000,8250.139048,0.000000,2
280,352894000,Reefer,1985.205000,4499.0000,4191.965616,0.000000,1
289,357735000,Reefer,6432.690000,7367.0000,6740.659925,0.000000,1
807,306723000,Unknown,1447.835000,264.0000,1517.151202,0.000000,1
837,366943720,Trollers,27.135000,31.5000,15.463735,1.603567,1
863,367379720,Pole_and_line,37.835000,63.0000,21.557292,3.207135,1
877,224072890,Unknown,29.350000,125.8450,7.043673,53.295203,1
1030,367366210,Pole_and_line,44.490000,110.5000,25.347056,5.879747,1
1114,353019000,Reefer,565.405000,4109.0000,494.534857,0.000000,1
1390,351747000,Reefer,6382.695000,7355.0000,6688.271376,0.000000,1


In [40]:
df.loc[(df["length_stddev"] > df["length"]/5),'length']=np.nan

In [42]:
# if stdv of tonnage > 20 percent the tonnage of the vessel, we should through out the length
df[(df["tonnage_stddev"] > df["tonnage"]/5.)]#['length']=np.nan

,mmsi,shiptype,length,tonnage,length_stddev,tonnage_stddev,number_shiptypes
3,224654000,Drifting_longlines,40.500000,434.223333,0.000000,89.719106,1
30,224135000,Drifting_longlines,31.850000,218.000000,2.910428,69.850260,1
42,412460299,Drifting_longlines,49.990000,514.000000,0.000000,105.736851,1
45,440733000,Drifting_longlines,47.210000,434.000000,0.000000,87.287156,1
63,440731000,Drifting_longlines,47.210000,429.750000,0.000000,86.850720,1
67,224623000,Drifting_longlines,55.000000,895.086667,0.000000,232.832348,1
93,306034000,Purse_seines,101.001429,2975.474286,7.780000,1898.698503,1
123,224047990,Drifting_longlines,29.900000,247.898000,3.815292,82.434840,1
129,440552000,Purse_seines,65.587500,1145.750000,3.070279,353.422591,1
136,503042740,Drifting_longlines,22.900000,96.666667,0.000000,24.019223,1


In [43]:
df.loc[(df["tonnage_stddev"] > df["tonnage"]/5),'tonnage']=np.nan

In [44]:
df.head()

,mmsi,shiptype,length,tonnage,length_stddev,tonnage_stddev,number_shiptypes
0,338793000,Purse_seines,63.12,1446.50,0.0,75.367670,1
1,416002947,Drifting_longlines,29.70,98.00,0.0,0.000000,1
2,416000961,Drifting_longlines,23.20,61.23,NaN,NaN,1
3,224654000,Drifting_longlines,40.50,NaN,0.0,89.719106,1
4,412371206,Drifting_longlines,43.20,560.00,0.0,0.000000,1


In [45]:
df.to_csv("input_lists/clav_v10_processed.csv")